# Building a Neural Network from Scratch

https://towardsdatascience.com/math-neural-network-from-scratch-in-python-d6da9f29ce65 

## what can go wrong if we don't do any preprocessing to the data before feeding it to the network?

first of all why does it matter?
the reason is related to gradient descent, when updating the parameters we multiply the difference between the true label and the predicted label by the input features. so the values of the input data contribute to the efficency of the learning. in 2 main ways:
1- if a value of the input is significantly large it will dominate the other feauters regardless of its relevancy
2- if all values are positive all the parameters will either increase or decrease, which would make the model go in zigzags to find the optimal value of the parameters which is not time efficent

#### main ways to tackle this problem is (Normalize and Standardize)
Normalization typically means rescales the values into a range of [0,1].

Standardization typically means rescales data to have a mean of 0 and a standard deviation of 1 (unit variance).




In [25]:
import numpy as np
# Normalization Function
def min_max_normalize(x):
    normalized_x = (x - np.min(x)) / (np.max(x) - np.min(x))
    return normalized_x

# Standardization Function
def standardize(x):
    standardized_x = ((x - np.mean(x)) / np.std(x))
    return standardized_x



### Abstract Base Class : Layer
The abstract class Layer, which all other layers will inherit from, handles simple properties which are an input, an output, and both a forward and backward methods.

In [26]:

# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In the abstract class above, backward_propagation function has an extra parameter, learning_rate, which is controlling the amount of learning/updating parameters using gradient descent.

### Backward Propagation
Suppose we have a matrix containing the derivative of the error with respect to that layer’s output: $\frac{\partial E}{\partial Y}$

We need :
- The derivative of the error with respect to the parameters ($\frac{\partial E}{\partial W}$, $\frac{\partial E}{\partial B}$)
- The derivative of the error with respect to the input ($\frac{\partial E}{\partial X}$)

Let's calculate $\frac{\partial E}{\partial W}$. This matrix should be the same size as $W$ itself : 

$i x j$ where $i$ is the number of input neurons and $j$ the number of output neurons. We need one gradient for every weight

### Coding the Fully Connected Layer

In [27]:
#from layer import Layer
import numpy as np

# inherit from base class Layer
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of edges that connects to neurons in next layer
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # dBias = output_error

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

### Activation Layer
All the calculation we did until now were completely linear, may not learn well. We need to add non-linearity to the model by applying non-linear functions to the output of some layers.

Now we need to redo the whole process for this new type of layer!

In [28]:
#from layer import Layer

# inherit from base class Layer
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

You can also write some activation functions and their derivatives in a separate file. These will be used later to create an ActivationLayer.

In [29]:
import numpy as np

# activation function and its derivative
def tanh(x):
    return np.tanh(x);

def tanh_prime(x):
    return 1-np.tanh(x)**2;

### Loss Function
Until now, for a given layer, we supposed that ∂E/∂Y was given (by the next layer). But what happens to the last layer? How does it get ∂E/∂Y? We simply give it manually, and it depends on how we define the error.
The error of the network, which measures how good or bad the network did for a given input data, is defined by you. 

There are many ways to define the error, and one of the most known is called MSE — Mean Squared Error.

In [30]:

import numpy as np

# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

### Network Class
Almost done ! We are going to make a Network class to create neural networks very easily using the building blocks we have prepared so far.


In [31]:
# example of a function for calculating softmax for a list of numbers
from numpy import exp
 
# calculate the softmax of a vector
def softmax(vector):
    e = exp(vector)
    return e / e.sum()

In [32]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

        
    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network 
    
    def fit(self, x_train, y_train, epochs, learning_rate):
        '''
        Fit function does the training. 
        Training data is passed 1-by-1 through the network layers during forward propagation.
        Loss (error) is calculated for each input and back propagation is performed via partial 
        derivatives on each layer.
        '''
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

### Building Neural Networks
Finally ! We can use our class to create a neural network with as many layers as we want ! We are going to build two neural networks : a simple XOR and a MNIST solver.


### Solve XOR
Starting with XOR is always important as it’s a simple way to tell if the network is learning anything at all.

In [33]:
import numpy as np

#from network import Network
#from fc_layer import FCLayer
#from activation_layer import ActivationLayer
#from activations import tanh, tanh_prime
#from losses import mse, mse_prime

# training data
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# network
net = Network()
net.add(FCLayer(2, 3))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(3, 1))
net.add(ActivationLayer(tanh, tanh_prime))

# train
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=1000, learning_rate=0.1)

# test
out = net.predict(x_train)
print(out)

epoch 1/1000   error=0.462480
epoch 2/1000   error=0.319540
epoch 3/1000   error=0.306289
epoch 4/1000   error=0.303070
epoch 5/1000   error=0.301432
epoch 6/1000   error=0.300202
epoch 7/1000   error=0.299116
epoch 8/1000   error=0.298090
epoch 9/1000   error=0.297091
epoch 10/1000   error=0.296105
epoch 11/1000   error=0.295123
epoch 12/1000   error=0.294139
epoch 13/1000   error=0.293150
epoch 14/1000   error=0.292152
epoch 15/1000   error=0.291143
epoch 16/1000   error=0.290121
epoch 17/1000   error=0.289083
epoch 18/1000   error=0.288029
epoch 19/1000   error=0.286958
epoch 20/1000   error=0.285867
epoch 21/1000   error=0.284756
epoch 22/1000   error=0.283624
epoch 23/1000   error=0.282471
epoch 24/1000   error=0.281296
epoch 25/1000   error=0.280099
epoch 26/1000   error=0.278880
epoch 27/1000   error=0.277638
epoch 28/1000   error=0.276374
epoch 29/1000   error=0.275089
epoch 30/1000   error=0.273783
epoch 31/1000   error=0.272458
epoch 32/1000   error=0.271113
epoch 33/1000   e

### Solve MNIST
We didn’t implemented the Convolutional Layer but this is not a problem. 
All we need to do is to reshape our data so that it can fit into a Fully Connected Layer.
MNIST Dataset consists of images of digits from 0 to 9, of shape 28x28x1. 
The goal is to predict what digit is drawn on a picture.

In [34]:
def evaluate(y, y_hat):
    y = np.reshape(y, y_hat.shape)
    confusion_matrix = np.zeros((y.shape[-1], y.shape[-1]))
    for i in range(y.shape[0]):
        prediction = np.argmax(y[i])
        true = np.argmax(y_hat[i])
        confusion_matrix[true][prediction] += 1
    diognal = []
    for i in range(confusion_matrix.shape[0]):
        diognal.append(confusion_matrix[i, i])
    precision_sum = np.sum(confusion_matrix, axis=0)
    recall_sum = np.sum(confusion_matrix, axis=1)
    all_samples = np.sum(recall_sum)
    accuracy = sum(diognal) / all_samples
    precision = np.average(diognal / precision_sum)
    recall = np.average(diognal / recall_sum)
    f1_score = 2 * precision * recall /  (precision + recall)
    print(f"accuracy: {accuracy}, precision: {precision}, recall: {recall}, f1_score: {f1_score}")


## without normalization or standardization 

In [39]:
import numpy as np

#from network import Network
#from fc_layer import FCLayer
#from activation_layer import ActivationLayer
#from activations import tanh, tanh_prime
#from losses import mse, mse_prime

from keras.datasets import mnist
from keras.utils import np_utils

# load MNIST from server
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
# x_train /= 255

# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = np_utils.to_categorical(y_train)

# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
# x_test /= 255

y_test = np_utils.to_categorical(y_test)

# Network
net = Network()
net.add(FCLayer(28*28, 100))                # input_shape=(1, 28*28)    ;   output_shape=(1, 100)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(100, 50))                   # input_shape=(1, 100)      ;   output_shape=(1, 50)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(50, 10))                    # input_shape=(1, 50)       ;   output_shape=(1, 10)
net.add(ActivationLayer(tanh, tanh_prime))

# train on 1000 samples
# as we didn't implemented mini-batch GD, training will be pretty slow if we update at each iteration on 60000 samples...
net.use(mse, mse_prime)
net.fit(x_train[:10000], y_train[:10000], epochs=35, learning_rate=0.1)

# test on 3 samples
# out = net.predict(x_test[0:3])
# print("\n")
# print("predicted values : ")
# print(out, end="\n")
# print("true values : ")
# print(y_test[0:3])

out = net.predict(x_test)
evaluate(out, y_test)

epoch 1/35   error=0.131684
epoch 2/35   error=0.109550
epoch 3/35   error=0.107744
epoch 4/35   error=0.107495
epoch 5/35   error=0.104120
epoch 6/35   error=0.100990
epoch 7/35   error=0.093961
epoch 8/35   error=0.096366
epoch 9/35   error=0.095718
epoch 10/35   error=0.093760
epoch 11/35   error=0.090286
epoch 12/35   error=0.090296
epoch 13/35   error=0.091627
epoch 14/35   error=0.094091
epoch 15/35   error=0.093387
epoch 16/35   error=0.091631
epoch 17/35   error=0.090042
epoch 18/35   error=0.088813
epoch 19/35   error=0.087787
epoch 20/35   error=0.085299
epoch 21/35   error=0.085386
epoch 22/35   error=0.086591
epoch 23/35   error=0.086287
epoch 24/35   error=0.083955
epoch 25/35   error=0.083122
epoch 26/35   error=0.085453
epoch 27/35   error=0.085122
epoch 28/35   error=0.085620
epoch 29/35   error=0.085533
epoch 30/35   error=0.085396
epoch 31/35   error=0.085245
epoch 32/35   error=0.084616
epoch 33/35   error=0.084402
epoch 34/35   error=0.084314
epoch 35/35   error=0.0

In [36]:
import numpy as np

#from network import Network
#from fc_layer import FCLayer
#from activation_layer import ActivationLayer
#from activations import tanh, tanh_prime
#from losses import mse, mse_prime

from keras.datasets import mnist
from keras.utils import np_utils

# load MNIST from server
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
# x_train /= 255
x_train = min_max_normalize(x_train)

# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = np_utils.to_categorical(y_train)

# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
# x_test /= 255
x_test = min_max_normalize(x_test)

y_test = np_utils.to_categorical(y_test)

# Network
net = Network()
net.add(FCLayer(28*28, 100))                # input_shape=(1, 28*28)    ;   output_shape=(1, 100)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(100, 50))                   # input_shape=(1, 100)      ;   output_shape=(1, 50)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(50, 10))                    # input_shape=(1, 50)       ;   output_shape=(1, 10)
net.add(ActivationLayer(tanh, tanh_prime))

# train on 1000 samples
# as we didn't implemented mini-batch GD, training will be pretty slow if we update at each iteration on 60000 samples...
net.use(mse, mse_prime)
net.fit(x_train[:10000], y_train[:10000], epochs=35, learning_rate=0.1)

# test on 3 samples
out = net.predict(x_test[0:3])
print("\n")
print("predicted values : ")
print(out, end="\n")
print("true values : ")
print(y_test[0:3])

out = net.predict(x_test)
evaluate(out, y_test)

epoch 1/35   error=0.088697
epoch 2/35   error=0.040496
epoch 3/35   error=0.029240
epoch 4/35   error=0.024331
epoch 5/35   error=0.021129
epoch 6/35   error=0.018673
epoch 7/35   error=0.016769
epoch 8/35   error=0.015315
epoch 9/35   error=0.014095
epoch 10/35   error=0.013062
epoch 11/35   error=0.012189
epoch 12/35   error=0.011478
epoch 13/35   error=0.010871
epoch 14/35   error=0.010317
epoch 15/35   error=0.009826
epoch 16/35   error=0.009390
epoch 17/35   error=0.008986
epoch 18/35   error=0.008645
epoch 19/35   error=0.008335
epoch 20/35   error=0.008050
epoch 21/35   error=0.007780
epoch 22/35   error=0.007528
epoch 23/35   error=0.007310
epoch 24/35   error=0.007123
epoch 25/35   error=0.006940
epoch 26/35   error=0.006755
epoch 27/35   error=0.006588
epoch 28/35   error=0.006430
epoch 29/35   error=0.006284
epoch 30/35   error=0.006151
epoch 31/35   error=0.006024
epoch 32/35   error=0.005899
epoch 33/35   error=0.005784
epoch 34/35   error=0.005671
epoch 35/35   error=0.0

In [37]:
import numpy as np

#from network import Network
#from fc_layer import FCLayer
#from activation_layer import ActivationLayer
#from activations import tanh, tanh_prime
#from losses import mse, mse_prime

from keras.datasets import mnist
from keras.utils import np_utils

# load MNIST from server
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
# x_train /= 255
x_train = standardize(x_train)

# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = np_utils.to_categorical(y_train)

# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
# x_test /= 255
x_test = standardize(x_test)

y_test = np_utils.to_categorical(y_test)

# Network
net = Network()
net.add(FCLayer(28*28, 100))                # input_shape=(1, 28*28)    ;   output_shape=(1, 100)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(100, 50))                   # input_shape=(1, 100)      ;   output_shape=(1, 50)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(50, 10))                    # input_shape=(1, 50)       ;   output_shape=(1, 10)
net.add(ActivationLayer(tanh, tanh_prime))

# train on 1000 samples
# as we didn't implemented mini-batch GD, training will be pretty slow if we update at each iteration on 60000 samples...
net.use(mse, mse_prime)
net.fit(x_train[:10000], y_train[:10000], epochs=35, learning_rate=0.1)

# test on 3 samples
out = net.predict(x_test[0:3])
print("\n")
print("predicted values : ")
print(out, end="\n")
print("true values : ")
print(y_test[0:3])

out = net.predict(x_test)
evaluate(out, y_test)

epoch 1/35   error=0.095880
epoch 2/35   error=0.045372
epoch 3/35   error=0.033885
epoch 4/35   error=0.027076
epoch 5/35   error=0.022866
epoch 6/35   error=0.019926
epoch 7/35   error=0.017762
epoch 8/35   error=0.015660
epoch 9/35   error=0.014109
epoch 10/35   error=0.012949
epoch 11/35   error=0.011903
epoch 12/35   error=0.011020
epoch 13/35   error=0.010465
epoch 14/35   error=0.009792
epoch 15/35   error=0.009307
epoch 16/35   error=0.008826
epoch 17/35   error=0.008460
epoch 18/35   error=0.008127
epoch 19/35   error=0.007796
epoch 20/35   error=0.007477
epoch 21/35   error=0.007268
epoch 22/35   error=0.007059
epoch 23/35   error=0.006830
epoch 24/35   error=0.006772
epoch 25/35   error=0.006547
epoch 26/35   error=0.006360
epoch 27/35   error=0.006203
epoch 28/35   error=0.006142
epoch 29/35   error=0.006006
epoch 30/35   error=0.005901
epoch 31/35   error=0.005769
epoch 32/35   error=0.005670
epoch 33/35   error=0.005631
epoch 34/35   error=0.005540
epoch 35/35   error=0.0

when we normalize our dataset the model gives better results